# see what angles get reconstructed well #

In [ ]:
import jkutils
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32" 
os.environ["PATH"] += os.pathsep + '/usr/local/cuda/bin/'
os.environ['PYTHONUNBUFFERED'] = '1'
import sys
import numpy as np
with jkutils.suppress_stdout_stderr(): #prevents printed info from theano
    import theano
    # theano.config.device = 'gpu'
    # theano.config.floatX = 'float32'
    import keras
from keras.models import Sequential, load_model
from keras.utils import np_utils
from keras.utils.io_utils import HDF5Matrix
from sklearn.model_selection import train_test_split
import h5py
import tables
import math
import resource
import shelve
import itertools

In [ ]:
    file_location = '/data/user/jkager/NN_Reco/johannes/updownclassification_2/'
    today = '2017-07-29'
    project_name = 'time10_h0_v1'
    project_folder = 'train_hist/{}/{}'.format(today, project_name)
    print "looking for", project_folder
    if not os.path.exists('{}'.format(os.path.join(file_location,project_folder))):
        print "project not found. exiting..."
        sys.exit(-1)
    shelf = shelve.open(os.path.join(file_location, project_folder, 'run_info.shlf'))
    input_files = shelf['Files']
    train_inds = shelf['Train_Inds'] 
    valid_inds = shelf['Valid_Inds']
    test_inds = shelf['Test_Inds']
    shelf.close()

    input_data, out_data, file_len = read_files(input_files.split(':'), data_location)
    print('\n Load the Model (final_network.h5) \n')
    model = load_model(os.path.join(\
    file_location,'train_hist/{}/{}/final_network.h5'.format(today, project_name)))
  
    print('\n Calculate Results... \n')
    res = []
    test_out = []
  
    for i in range(len(input_data)):
        print('Predict Values for {}'.format(input_files[i]))
        test_in_chunk  = np.array(map(np.ndarray.flatten, preprocess(input_data[i][test_inds[i][0]:test_inds[i][1]])))
        test_out_chunk = zenith_to_binary(out_data[i][test_inds[i][0]:test_inds[i][1],zenith:zenith+1])
        res_chunk = model.predict(test_in_chunk, verbose=int(parser.get('Training_Parameters', 'verbose')))
        res.extend(list(res_chunk))
        test_out.extend(list(test_out_chunk))
  
    res = np.squeeze(res)
    test_out = np.squeeze(test_out)
    np.save(os.path.join(file_location,'train_hist/{}/{}/test_results.npy'.format(today, project_name)), 
      [res, test_out])
    correct = np.sum(res == test_out)
    total = len(res)
    print "{} / {} = {:6.2f}%".format(correct, total, float(correct)/total*100)
  
    print(' \n Finished .... ')